# Credmark Modeling Framework Example for Jupyter notebook
## Load/Create/Reload Model

version: 2022.6.12

In [ ]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext

from credmark.dto import *
from credmark.cmf.types import *

from credmark.cmf.engine.dev_models.console import get_dt, get_block, log_output

## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>

In [ ]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': 14911537,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
          'api_url': None,
          'run_id': None,
          'console': True,
          'use_local_models': None # Or, '*', or '-', or 'model_to_be_run_locally'
         }
context = EngineModelContext.create_context(**params)

### 1. Reload model

If we made changes to models on disk in the `models` directory, we can call `context.reload_model()` to refresh the cache.

In [ ]:
context.reload_model()

### 2. Define/Add/Remove/Replace model on-the-fly

1. Define the model with a class for BTC/USD feed

In [ ]:
from credmark.cmf.model import Model
from credmark.cmf.model.errors import ModelDataError, ModelRunError

In [ ]:
@Model.describe(
    slug='jit.feed',
    version='1.0',
    input=EmptyInput,
    output=dict)
class JitFeed(Model):
    def run(self, input):
        feedreg = self.context.models.chainlink.get_feed_registry(return_type=Contract)
        feed = feedreg.functions.getFeed(Address('0xbBbBBBBbbBBBbbbBbbBbbbbBBbBbbbbBbBbbBBbB'),
                                         Address('0x0000000000000000000000000000000000000348')).call()
        return {'feed': feed}

2. Add/Run/Remove the model

- add model, `context.add_model(mclass)`
- remove model, `context.remove_model_by_slug(model_slug)`

In [ ]:
context.add_model(JitFeed)

# Check whether the model has been loaded
'jit.feed' in model_loader.loaded_model_version_lists()

In [ ]:
context.models.jit.feed?

In [ ]:
context.models.jit.feed()

In [ ]:
context.remove_model_by_slug('jit.feed')

In [ ]:
'jit.feed' in model_loader.loaded_model_version_lists()

3. Update the model to ETH/USD feed and again

In [ ]:
@Model.describe(
    slug='jit.feed',
    version='1.0',
    input=EmptyInput,
    output=dict)
class JitFeed(Model):
    def run(self, input):
        feedreg = self.context.models.chainlink.get_feed_registry(return_type=Contract)
        feed = feedreg.functions.getFeed(Address('0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'),
                                         Address('0x0000000000000000000000000000000000000348')).call()
        return {'feed': feed}

In [ ]:
context.add_model(JitFeed)

In [ ]:
context.models.jit.feed()

4. Add another model by using the previously-defined `jit.feed` model

In [ ]:
@Model.describe(
    slug='jit.feed-price',
    version='1.0',
    input=EmptyInput,
    output=dict)
class JitFeedWithPrice(Model):
    def run(self, input):
        feed = self.context.models.jit.feed(input)['feed']
        cc = Contract(address=Address(feed))
        price = self.context.models.chainlink.price_by_feed(cc, return_type=Price).price
        return {'price': price}

In [ ]:
context.add_model(JitFeedWithPrice)

In [ ]:
context.models(get_block(get_dt(2021, 7, 28))).jit.feed_price()

In [ ]:
context.models(get_block(get_dt(2021, 7, 28))).jit.feed()